## Project Summary V5 — Dual-Factor VIX Regime Classification

This V5 notebook upgrades the V4 framework with a sophisticated VIX regime system:

- **Dual-factor regime**: combines VIX time-proportion (noise filter) with VIX/VIX3M term structure inversion
- **Three regimes**: Normal (full exposure), Elevated (reduced), Panic (minimal)
- **Adaptive thresholds**: rolling 252-day 75th percentile with absolute floor of 20
- **Graceful fallback**: degrades safely when VIX3M data unavailable
- Retains V4 improvements: conviction weighting, conditional short leg, lower momentum thresholds

### Data Collection

In this project, we use three datasets: adj_close.csv (Yahoo Finance API), volume.csv (Yahoo Finance API), and VIX.csv proxy from `^VIX` and `^VIX3M` (3-month volatility, for term structure) in Yahoo Finance, with IRX (`^IRX`) for cash return proxy. Benchmarks include both **XLK** and **SPY**.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import time
import requests

# load environment (for any other API keys you may have)
from dotenv import load_dotenv
load_dotenv()

# 统一使用 Yahoo Finance API 获取数据


In [ ]:
# ============================================================
# UNIFIED DATA FETCHER - Yahoo Finance API (Direct Requests)
# ============================================================
# No external API keys required, no rate limits like Tiingo
# Fetches all stock data, VIX, and IRX equivalent (^IRX) in one place

import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime

def fetch_yahoo_data(ticker, start_dt, end_dt, headers):
    """
    Fetch historical data for a single ticker from Yahoo Finance API.
    Returns tuple of (adj_close_series, volume_series) or (None, None) if failed.
    """
    start_ts = int(start_dt.timestamp())
    end_ts = int(end_dt.timestamp())
    
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?period1={start_ts}&period2={end_ts}&interval=1d'
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        data = response.json()
        
        if 'chart' in data and 'result' in data['chart'] and data['chart']['result']:
            result = data['chart']['result'][0]
            timestamps = result.get('timestamp', [])
            
            if timestamps and len(timestamps) > 10:
                quote = result['indicators']['quote'][0]
                adjclose = result['indicators'].get('adjclose', [{}])[0].get('adjclose', quote.get('close', []))
                vol = quote.get('volume', [])
                
                # Create date index (normalized to remove time component)
                dates = pd.to_datetime(timestamps, unit='s').normalize()
                
                # Create Series
                adj_close_series = pd.Series(adjclose, index=dates, name=ticker)
                volume_series = pd.Series(vol, index=dates, name=ticker)
                
                # Ensure timezone-naive
                if adj_close_series.index.tz is not None:
                    adj_close_series.index = adj_close_series.index.tz_localize(None)
                if volume_series.index.tz is not None:
                    volume_series.index = volume_series.index.tz_localize(None)
                
                # Remove duplicates
                adj_close_series = adj_close_series[~adj_close_series.index.duplicated(keep='first')]
                volume_series = volume_series[~volume_series.index.duplicated(keep='first')]
                
                return adj_close_series, volume_series
        
        return None, None
        
    except Exception as e:
        print(f"    Error: {str(e)[:50]}")
        return None, None

# ============================================================
# CONFIGURATION
# ============================================================

# XLK constituents + benchmark ETF + VIX + IRX
stock_tickers = [
    'AAPL', 'ACN', 'ADBE', 'ADI', 'AKAM', 'AMD', 'AMAT', 'ANET', 'APH', 'AVGO',
    'CDNS', 'CDW', 'CRWD', 'CRM', 'CSCO', 'CTSH', 'DDOG', 'DELL', 'ENPH', 'EPAM',
    'FICO', 'FFIV', 'FSLR', 'FTNT', 'GEN', 'GDDY', 'GLW', 'HPE', 'HPQ', 'IBM',
    'INTC', 'INTU', 'IT', 'JBL', 'KEYS', 'KLAC', 'LRCX', 'MCHP', 'MPWR', 'MSI',
    'MSFT', 'MU', 'NOW', 'NVDA', 'NXPI', 'ON', 'ORCL', 'PANW', 'PLTR', 'PTC',
    'QCOM', 'ROP', 'SMCI', 'SNPS', 'STX', 'SWKS', 'TEL', 'TER', 'TDY', 'TXN',
    'TYL', 'TRMB', 'VRSN', 'WDC', 'WDAY', 'ZBRA'
]

# Add benchmark and market indicators
benchmark_tickers = ['XLK', 'SPY']           # XLK ETF benchmark
market_indicators = ['^VIX', '^VIX3M', '^IRX']  # VIX and 13-week T-bill rate

# Combine all tickers
all_tickers = stock_tickers + benchmark_tickers + market_indicators

# Date range
start_dt = datetime(2019, 12, 1)
end_dt = datetime(2026, 1, 30)

# HTTP headers
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

print("=" * 70)
print("📊 UNIFIED DATA FETCHER - Yahoo Finance API")
print("=" * 70)
print(f"📅 Date range: {start_dt.date()} to {end_dt.date()}")
print(f"📈 Total tickers to fetch: {len(all_tickers)}")
print(f"   - Stocks: {len(stock_tickers)}")
print(f"   - Benchmark: {benchmark_tickers}")
print(f"   - Market Indicators: {market_indicators}")
print("-" * 70)

# ============================================================
# FETCH ALL DATA
# ============================================================

adj_close_data = {}
volume_data = {}
failed_tickers = []

for i, ticker in enumerate(all_tickers):
    print(f"[{i+1}/{len(all_tickers)}] Downloading {ticker}...", end=" ")
    
    adj_series, vol_series = fetch_yahoo_data(ticker, start_dt, end_dt, headers)
    
    if adj_series is not None and len(adj_series) > 100:
        adj_close_data[ticker] = adj_series
        volume_data[ticker] = vol_series
        valid_count = adj_series.notna().sum()
        print(f"✅ {valid_count} rows")
    else:
        print(f"❌ Failed or insufficient data")
        failed_tickers.append(ticker)
    
    # Small delay to be respectful to Yahoo's servers
    time.sleep(0.3)

# ============================================================
# CREATE DATAFRAMES
# ============================================================

print("\n" + "-" * 70)
print("📊 Creating unified DataFrames...")

# Create DataFrames
adj_close = pd.DataFrame(adj_close_data)
volume = pd.DataFrame(volume_data)

# Ensure datetime index
adj_close.index = pd.to_datetime(adj_close.index)
volume.index = pd.to_datetime(volume.index)

# Sort by date
adj_close = adj_close.sort_index()
volume = volume.sort_index()

# Forward fill for short gaps (limit 5 days to avoid long gaps)
adj_close = adj_close.ffill(limit=5)
volume = volume.ffill(limit=5)

# Remove VIX and IRX from volume (they don't have volume data)
for indicator in ['^VIX', '^IRX']:
    if indicator in volume.columns:
        volume = volume.drop(columns=[indicator])

# ============================================================
# SAVE DATA
# ============================================================

adj_close.to_csv('adj_close.csv')
volume.to_csv('volume.csv')

# ============================================================
# SUMMARY
# ============================================================

print("\n" + "=" * 70)
print("✅ DATA DOWNLOAD COMPLETE!")
print("=" * 70)
print(f"📊 Successfully downloaded: {len(adj_close.columns)} tickers")
print(f"❌ Failed: {len(failed_tickers)} tickers")
if failed_tickers:
    print(f"   Failed list: {failed_tickers}")
print(f"\n📅 Date range: {adj_close.index[0].strftime('%Y-%m-%d')} to {adj_close.index[-1].strftime('%Y-%m-%d')}")
print(f"📈 Total trading days: {len(adj_close)}")

# Check key indicators
print(f"\n📊 Key Indicators Status:")
for indicator in ['XLK', '^VIX', '^IRX']:
    if indicator in adj_close.columns:
        count = adj_close[indicator].notna().sum()
        latest = adj_close[indicator].dropna().iloc[-1] if count > 0 else 'N/A'
        print(f"   {indicator}: {count} valid rows, latest value = {latest:.4f}" if isinstance(latest, (int, float)) else f"   {indicator}: {count} valid rows")
    else:
        print(f"   {indicator}: ❌ Not available")

print(f"\n💾 Data saved to:")
print(f"   - adj_close.csv ({adj_close.shape})")
print(f"   - volume.csv ({volume.shape})")

In [ ]:
# ============================================================
# DATA VERIFICATION & QUALITY CHECK
# ============================================================

# Reload to verify
adj_close = pd.read_csv('adj_close.csv', index_col=0, parse_dates=True)
volume = pd.read_csv('volume.csv', index_col=0, parse_dates=True)

print("=" * 70)
print("📊 DATA QUALITY VERIFICATION")
print("=" * 70)

# Overall statistics
print(f"\n1 Overall Statistics:")
print(f"   Adjusted Close: {adj_close.shape[0]} rows × {adj_close.shape[1]} columns")
print(f"   Volume: {volume.shape[0]} rows × {volume.shape[1]} columns")
print(f"   Date range: {adj_close.index[0].strftime('%Y-%m-%d')} to {adj_close.index[-1].strftime('%Y-%m-%d')}")

# VIX statistics
print(f"\n2 VIX Data:")
if '^VIX' in adj_close.columns:
    vix = adj_close['^VIX'].dropna()
    print(f"   Valid rows: {len(vix)}")
    print(f"   Range: {vix.min():.2f} - {vix.max():.2f}")
    print(f"   Mean: {vix.mean():.2f}, Std: {vix.std():.2f}")
    print(f"   Latest: {vix.iloc[-1]:.2f}")
else:
    print("   ❌ VIX not available")

# IRX statistics (13-week T-bill rate)
print(f"\n3 IRX Data (Risk-free rate proxy):")
if '^IRX' in adj_close.columns:
    irx = adj_close['^IRX'].dropna()
    print(f"   Valid rows: {len(irx)}")
    print(f"   Range: {irx.min():.3f}% - {irx.max():.3f}%")
    print(f"   Mean: {irx.mean():.3f}%, Std: {irx.std():.3f}%")
    print(f"   Latest: {irx.iloc[-1]:.3f}%")
else:
    print("   ❌ IRX not available")

# XLK statistics
print(f"\n4 XLK Benchmark:")
if 'XLK' in adj_close.columns:
    xlk = adj_close['XLK'].dropna()
    print(f"   Valid rows: {len(xlk)}")
    print(f"   Price range: ${xlk.min():.2f} - ${xlk.max():.2f}")
    print(f"   Latest: ${xlk.iloc[-1]:.2f}")
else:
    print("   ❌ XLK not available")

# VIX3M statistics
print(f"\n5 VIX3M Data:")
if '^VIX3M' in adj_close.columns:
    vix3m = adj_close['^VIX3M'].dropna()
    print(f"   Valid rows: {len(vix3m)}")
    print(f"   Range: {vix3m.min():.2f} - {vix3m.max():.2f}")
    print(f"   Latest: {vix3m.iloc[-1]:.2f}")
else:
    print("   VIX3M not available - will use single-factor regime")

# SPY statistics
print(f"\n6 SPY Benchmark:")
if 'SPY' in adj_close.columns:
    spy = adj_close['SPY'].dropna()
    print(f"   Valid rows: {len(spy)}")
    print(f"   Price range: ${spy.min():.2f} - ${spy.max():.2f}")
    print(f"   Latest: ${spy.iloc[-1]:.2f}")
else:
    print("   ❌ SPY not available")

# Missing data check
print(f"\n7 Missing Data Check:")
missing_pct = (adj_close.isna().sum() / len(adj_close) * 100).sort_values(ascending=False)
high_missing = missing_pct[missing_pct > 5]
if len(high_missing) > 0:
    print(f"   ⚠️ Tickers with >5% missing data:")
    for ticker, pct in high_missing.items():
        print(f"      {ticker}: {pct:.1f}%")
else:
    print("   ✅ All tickers have <5% missing data")

print(f"\n✅ Data verification complete!")

### Momentum calculation methods

Calculate_momentum_scores, is designed to compute momentum scores for a universe of stocks based on historical price and volume data. It offers three distinct calculation methods: 'simple' (price change percentage), 'risk_adjusted' (momentum divided by annualized volatility), and 'volume_weighted' (momentum multiplied by a relative volume factor). 

In [ ]:
def calculate_momentum_scores(adj_close, volume, daily_returns, previous_date, lookback_months, method='simple'):
    """
    Calculates momentum scores for all valid stocks on a given date.
    
    Parameters:
    - adj_close: DataFrame of adjusted close prices (loaded from the CSV )
    - volume: DataFrame of trading volumes
    - daily_returns: DataFrame of daily returns (adj_close.pct_change())
    - previous_date: The date for calculation (end of the previous month, pd.Timestamp)
    - lookback_months: The lookback period in months (e.g., 3, 6, 12)
    - method: 'simple', 'risk_adjusted', or 'volume_weighted'
    
    Returns:
    - A Series of scores indexed by ticker (higher is better momentum)
    
    Example usage:
    scores = calculate_momentum_scores(adj_close, volume, daily_returns, pd.Timestamp('2025-07-31'), 6, 'simple')
    """
    
    # Handle timezone issues - unify by removing timezone information
    if adj_close.index.tz is not None:
        adj_close_tz_naive = adj_close.copy()
        adj_close_tz_naive.index = adj_close.index.tz_localize(None)
    else:
        adj_close_tz_naive = adj_close
    
    if volume.index.tz is not None:
        volume_tz_naive = volume.copy()
        volume_tz_naive.index = volume.index.tz_localize(None)
    else:
        volume_tz_naive = volume
    
    if daily_returns.index.tz is not None:
        daily_returns_tz_naive = daily_returns.copy()
        daily_returns_tz_naive.index = daily_returns.index.tz_localize(None)
    else:
        daily_returns_tz_naive = daily_returns
    
    # Ensure previous_date is also timezone-naive
    if hasattr(previous_date, 'tz') and previous_date.tz is not None:
        previous_date = previous_date.tz_localize(None)
    
    # Find the lookback start date (approximately N months ago, taking the closest trading day)
    lookback_start = previous_date - pd.DateOffset(months=lookback_months)
    available_dates = adj_close_tz_naive.index[adj_close_tz_naive.index >= lookback_start]
    if len(available_dates) == 0:
        print(f"Warning: No data found after {lookback_start}")
        return pd.Series(dtype=float)
    lookback_start = available_dates[0]
    
    # Extract lookback period data, including only stocks with no NaN values
    close_lookback = adj_close_tz_naive.loc[lookback_start:previous_date]
    
    # Exclude indices and ETFs, keeping only individual stocks
    exclude_symbols = ['XLK', 'SPY', '^VIX', '^VIX3M', '^IRX']
    valid_stocks = []
    for col in close_lookback.columns:
        if col not in exclude_symbols and close_lookback[col].notna().all():
            valid_stocks.append(col)
    
    valid_stocks = pd.Index(valid_stocks)
    
    if len(valid_stocks) < 10:  # Minimum stock count threshold to avoid invalid calculations
        print(f"Warning: Too few valid stocks ({len(valid_stocks)}) for effective calculation")
        return pd.Series(dtype=float)
    
    # Basic momentum: price rate of change
    try:
        close_t = adj_close_tz_naive.loc[previous_date, valid_stocks]
        close_tk = adj_close_tz_naive.loc[lookback_start, valid_stocks]
        mom = close_t / close_tk - 1
    except KeyError as e:
        print(f"Error: Date {previous_date} or {lookback_start} not in data")
        return pd.Series(dtype=float)
    
    if method == 'simple':
        scores = mom
    elif method == 'risk_adjusted':
        # Calculate annualized volatility with better error handling
        daily_ret_lb = daily_returns_tz_naive.loc[lookback_start:previous_date, valid_stocks]
        vol = daily_ret_lb.std() * np.sqrt(252)  # Annualize
        
        # Replace zeros and very small volatilities to avoid extreme ratios
        vol = vol.replace(0, np.nan)
        vol = vol.where(vol > 0.01, np.nan)  # Filter out extremely low volatility stocks
        
        scores = mom / vol
        # Filter out extreme values
        scores = scores.where(np.abs(scores) <= 10, np.nan)  # Cap at reasonable range
        
    elif method == 'volume_weighted':
        # Baseline volume: past 12 months or available period
        baseline_start = previous_date - pd.DateOffset(months=12)
        available_baseline_dates = adj_close_tz_naive.index[adj_close_tz_naive.index >= baseline_start]
        if len(available_baseline_dates) == 0:
            baseline_start = adj_close_tz_naive.index[0]  # Use the earliest available date
        else:
            baseline_start = available_baseline_dates[0]
            
        # Select only stocks that exist in the volume data
        volume_valid_stocks = valid_stocks.intersection(volume_tz_naive.columns)
        
        vol_lb = volume_tz_naive.loc[lookback_start:previous_date, volume_valid_stocks].mean()
        vol_baseline = volume_tz_naive.loc[baseline_start:previous_date, volume_valid_stocks].mean()
        
        # Avoid division by zero in volume calculations
        vol_baseline = vol_baseline.replace(0, np.nan)
        volume_factor = vol_lb / vol_baseline  # Relative volume
        
        # Cap volume factor to reasonable range
        volume_factor = volume_factor.where((volume_factor >= 0.1) & (volume_factor <= 10), 1.0)
        
        # Calculate scores only for stocks with volume data
        scores = pd.Series(index=valid_stocks, dtype=float)
        for stock in volume_valid_stocks:
            if stock in mom.index and not pd.isna(volume_factor[stock]) and volume_factor[stock] > 0:
                scores[stock] = mom[stock] * volume_factor[stock]
            elif stock in mom.index:
                scores[stock] = mom[stock]  # If no volume data, use basic momentum
        
        scores = scores.dropna()
    else:
        raise ValueError(f"Invalid method: {method}")
    
    # Final cleanup: remove infinite values and extreme outliers
    scores = scores.replace([np.inf, -np.inf], np.nan)
    
    # Filter out extreme outliers (beyond 3 standard deviations)
    if len(scores.dropna()) > 5:
        mean_score = scores.mean()
        std_score = scores.std()
        if std_score > 0:
            scores = scores.where(np.abs(scores - mean_score) <= 3 * std_score, np.nan)
    
    return scores.dropna()  # Drop any remaining NaNs

In [ ]:
# Reload data
adj_close = pd.read_csv('adj_close.csv', index_col=0, parse_dates=True)
volume = pd.read_csv('volume.csv', index_col=0, parse_dates=True)

# Handle timezone issues - unify by removing timezone information
if adj_close.index.tz is not None:
    adj_close.index = adj_close.index.tz_localize(None)
if volume.index.tz is not None:
    volume.index = volume.index.tz_localize(None)

# Fix FutureWarning - explicitly specify fill_method=None
daily_returns = adj_close.pct_change(fill_method=None)

print("Data loading complete:")
print(f"adj_close shape: {adj_close.shape}")
print(f"volume shape: {volume.shape}")
print(f"Date range: {adj_close.index[0]} to {adj_close.index[-1]}")

# Test case: End of July 2025, 6-month lookback, simple method
test_date = pd.Timestamp('2026-01-30')  # Assume this is the month-end; if no data, take the nearest

# Ensure test_date is timezone-naive
if hasattr(test_date, 'tz') and test_date.tz is not None:
    test_date = test_date.tz_localize(None)

if test_date not in adj_close.index:
    available_dates = adj_close.index[adj_close.index <= test_date]
    if len(available_dates) > 0:
        test_date = available_dates[-1]  # Take the most recent trading day
    else:
        test_date = adj_close.index[-1]  # If no suitable date, take the latest date

print(f"\nUsing test date: {test_date}")

# Simple Momentum Scores
print("\n=== Simple Momentum Scores ===")
scores_simple = calculate_momentum_scores(adj_close, volume, daily_returns, test_date, 6, 'simple')
if len(scores_simple) > 0:
    print(f"Simple momentum scores (top 5):\n{scores_simple.sort_values(ascending=False).head(5)}")
else:
    print("Simple momentum calculation failed")

# Risk-Adjusted Momentum Scores
print("\n=== Risk-Adjusted Momentum Scores ===")
scores_risk = calculate_momentum_scores(adj_close, volume, daily_returns, test_date, 6, 'risk_adjusted')
if len(scores_risk) > 0:
    print(f"Risk-adjusted momentum scores (top 5):\n{scores_risk.sort_values(ascending=False).head(5)}")
else:
    print("Risk-adjusted momentum calculation failed")

# Volume-Weighted Momentum Scores
print("\n=== Volume-Weighted Momentum Scores ===")
scores_vol = calculate_momentum_scores(adj_close, volume, daily_returns, test_date, 6, 'volume_weighted')
if len(scores_vol) > 0:
    print(f"Volume-weighted momentum scores (top 5):\n{scores_vol.sort_values(ascending=False).head(5)}")
else:
    print("Volume-weighted momentum calculation failed")

print(f"\n=== Test Complete ===")
print(f"Number of valid stocks: {len(scores_simple)} (simple), {len(scores_risk)} (risk-adjusted), {len(scores_vol)} (volume-weighted)")


### V5 Dual-Factor VIX Regime Classification

V5 replaces the V4 single-point VIX gating with a dual-factor monthly regime classifier:

**Factor 1 — Time-Proportion (Noise Filter)**
- Threshold: `thr = max(20, rolling_252d_75th_percentile(VIX))`
- Metric: proportion of trading days in month where VIX > thr
- Trigger: proportion > 25%

**Factor 2 — Term Structure Inversion (VIX/VIX3M)**
- Normal (contango): VIX < VIX3M, ratio < 1
- Elevated: monthly avg ratio > 0.97
- Panic (backwardation): monthly avg ratio > 1.00

**Three Regimes:**

| Regime | Condition | w_vix |
|--------|-----------|-------|
| Normal | Neither factor triggers | 1.00 |
| Elevated | Exactly one factor triggers | 0.75 |
| Panic | Both factors trigger (high time-proportion AND term structure inversion) | 0.35 |

**Fallback**: If VIX3M unavailable, max regime is Elevated (no Panic).

Position sizing formula (unchanged from V4):
`equity_weight = w_vix * (0.40 * w_mom + 0.35 * w_trend + 0.25 * 1.0)`

In [ ]:
def classify_trend(adj_close, signal_date):
    if 'XLK' not in adj_close.columns:
        return 'Mixed'
    idx = adj_close.index[adj_close.index <= signal_date]
    if len(idx) == 0:
        return 'Mixed'
    sdate = idx[-1]

    def lb_ret(months):
        start = sdate - pd.DateOffset(months=months)
        prior_idx = adj_close.index[adj_close.index <= start]
        if len(prior_idx) == 0:
            return np.nan
        p0 = adj_close.at[prior_idx[-1], 'XLK']
        p1 = adj_close.at[sdate, 'XLK']
        if pd.isna(p0) or pd.isna(p1) or p0 == 0:
            return np.nan
        return p1 / p0 - 1

    r3 = lb_ret(3)
    r6 = lb_ret(6)
    r12 = lb_ret(12)
    positives = int(r3 > 0) + int(r6 > 0) + int(r12 > 0)
    if positives == 3:
        return 'Strong Uptrend'
    if positives == 2:
        return 'Uptrend'
    if positives == 1:
        return 'Mixed'
    return 'Downtrend'


def calculate_position_size_v3(vix_level, momentum_signal, trend_strength):
    if pd.isna(vix_level):
        w_vix = 0.75
    elif vix_level < 15:
        w_vix = 1.0
    elif vix_level < 20:
        w_vix = 1.0
    elif vix_level < 25:
        w_vix = 0.75
    elif vix_level < 30:
        w_vix = 0.50
    elif vix_level < 40:
        w_vix = 0.25
    else:
        w_vix = 0.0

    if pd.isna(momentum_signal):
        w_mom = 0.50
    elif momentum_signal > 1.5:
        w_mom = 1.0
    elif momentum_signal > 1.0:
        w_mom = 0.85
    elif momentum_signal > 0.5:
        w_mom = 0.70
    elif momentum_signal > 0.0:
        w_mom = 0.50
    else:
        w_mom = 0.25

    trend_map = {
        'Strong Uptrend': 1.0,
        'Uptrend': 0.80,
        'Mixed': 0.50,
        'Downtrend': 0.25,
    }
    w_trend = trend_map.get(trend_strength, 0.50)

    equity_weight = w_vix * (0.4 * w_mom + 0.4 * w_trend + 0.2 * 1.0)
    equity_weight = float(np.clip(equity_weight, 0.0, 1.0))
    cash_weight = 1.0 - equity_weight

    return {
        'equity_weight': equity_weight,
        'cash_weight': cash_weight,
        'signal_details': {
            'vix_level': float(vix_level) if pd.notna(vix_level) else np.nan,
            'momentum_signal': float(momentum_signal) if pd.notna(momentum_signal) else np.nan,
            'trend_strength': trend_strength,
            'w_vix': float(w_vix),
            'w_mom': float(w_mom),
            'w_trend': float(w_trend),
        },
    }


def compute_monthly_vix_regime(
    vix_daily,
    vix3m_daily,
    signal_date,
    lookback_days=252,
    q=0.75,
    vix_floor=20.0,
    p_thr=0.25,
    r_elev=0.97,
    r_panic=1.00,
    min_days=10,
    prev_regime='Normal',
):
    """
    Dual-factor monthly VIX regime classifier.

    Factor 1: Time-proportion of days VIX > adaptive threshold
    Factor 2: VIX/VIX3M term structure ratio

    Returns (regime_str, w_vix_float)
    """
    w_map = {'Normal': 1.0, 'Elevated': 0.75, 'Panic': 0.35}

    vix_upto = vix_daily.loc[:signal_date].dropna()
    if len(vix_upto) < 30:
        return prev_regime, w_map.get(prev_regime, 1.0)

    # Adaptive threshold: rolling 252d 75th percentile with floor
    window = vix_upto.iloc[-lookback_days:] if len(vix_upto) > lookback_days else vix_upto
    thr_vix = max(vix_floor, float(window.quantile(q)))

    # This month's daily VIX
    month_period = signal_date.to_period('M')
    month_mask = vix_upto.index.to_period('M') == month_period
    vix_month = vix_upto.loc[month_mask]

    if len(vix_month) < min_days:
        return prev_regime, w_map.get(prev_regime, 1.0)

    # Factor 1: time-proportion
    p_high = float((vix_month > thr_vix).mean())
    high_time = (p_high > p_thr)

    # Factor 2: term structure ratio
    r_avg = None
    if vix3m_daily is not None and len(vix3m_daily) > 0:
        vix3m_upto = vix3m_daily.loc[:signal_date].dropna()
        common = vix_month.index.intersection(vix3m_upto.index)
        if len(common) >= min_days:
            ratio = (vix_month.loc[common] / vix3m_upto.loc[common])
            ratio = ratio.replace([np.inf, -np.inf], np.nan).dropna()
            if len(ratio) >= min_days:
                r_avg = float(ratio.mean())

    # Classify regime
    if r_avg is None:
        # Fallback: no VIX3M data, cap at Elevated
        regime = 'Elevated' if high_time else 'Normal'
    else:
        inv_panic = (r_avg > r_panic)
        flat_elev = (r_avg > r_elev)

        if high_time and inv_panic:
            regime = 'Panic'
        elif high_time or flat_elev:
            regime = 'Elevated'
        else:
            regime = 'Normal'

    return regime, w_map[regime]


def calculate_position_size_v5(regime, w_vix, momentum_signal, trend_strength):
    """V5 position sizing using regime-based VIX weight."""
    # Momentum signal weight (same thresholds as V4)
    if pd.isna(momentum_signal):
        w_mom = 0.50
    elif momentum_signal > 0.8:
        w_mom = 1.0
    elif momentum_signal > 0.4:
        w_mom = 0.90
    elif momentum_signal > 0.0:
        w_mom = 0.75
    elif momentum_signal > -0.3:
        w_mom = 0.50
    else:
        w_mom = 0.20

    trend_map = {
        'Strong Uptrend': 1.0,
        'Uptrend': 0.80,
        'Mixed': 0.50,
        'Downtrend': 0.25,
    }
    w_trend = trend_map.get(trend_strength, 0.50)

    # Same formula as V4 but w_vix comes from regime classifier
    equity_weight = w_vix * (0.40 * w_mom + 0.35 * w_trend + 0.25 * 1.0)
    equity_weight = float(np.clip(equity_weight, 0.0, 1.0))
    cash_weight = 1.0 - equity_weight

    return {
        'equity_weight': equity_weight,
        'cash_weight': cash_weight,
        'signal_details': {
            'regime': regime,
            'w_vix': float(w_vix),
            'momentum_signal': float(momentum_signal) if pd.notna(momentum_signal) else np.nan,
            'trend_strength': trend_strength,
            'w_mom': float(w_mom),
            'w_trend': float(w_trend),
        },
    }

### Strategy Backtesting: V5 and V4 Baseline

In [ ]:
def backtest_strategy_v3(
    adj_close,
    volume,
    lookback_months=6,
    method='simple',
    decile=0.3,
    tc_bps=5,
    holding_period=1,
    winsor_q=0.01,
):
    daily_returns = adj_close.pct_change(fill_method=None)

    month_ends = adj_close.resample('M').last().index
    valid_month_ends = []
    for me in month_ends:
        month_data = adj_close[(adj_close.index.month == me.month) & (adj_close.index.year == me.year)]
        if len(month_data) > 0:
            valid_month_ends.append(month_data.index[-1])
    valid_month_ends = pd.DatetimeIndex(valid_month_ends).unique().sort_values()

    if len(valid_month_ends) <= lookback_months + 1:
        return pd.DataFrame(columns=[
            'strategy_returns', 'equity_returns', 'cash_returns',
            'equity_weight', 'cash_weight', 'num_holdings', 'turnover'
        ])

    def _get_ret(symbol, start_date, end_date):
        if symbol not in adj_close.columns:
            return np.nan
        try:
            p0 = adj_close.at[start_date, symbol]
            p1 = adj_close.at[end_date, symbol]
            if pd.isna(p0) or pd.isna(p1) or p0 == 0:
                return np.nan
            return p1 / p0 - 1
        except Exception:
            return np.nan

    def _cash_return(prev_date, curr_date):
        if '^IRX' not in adj_close.columns:
            return 0.0
        rf = adj_close['^IRX'].loc[(adj_close.index > prev_date) & (adj_close.index <= curr_date)]
        if len(rf) == 0:
            return 0.0
        daily_rf = pd.to_numeric(rf, errors='coerce') / 100.0 / 252.0
        daily_rf = daily_rf.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        return float((1.0 + daily_rf).prod() - 1.0)

    out_rows = []
    out_index = []

    current_holdings = pd.Index([])
    equity_weight = 0.0
    cash_weight = 1.0
    prev_asset_weights = {'CASH': 1.0}

    start_i = lookback_months
    for i in range(start_i, len(valid_month_ends)):
        current_date = valid_month_ends[i]
        previous_date = valid_month_ends[i - 1]

        rebalance_now = ((i - start_i) % holding_period == 0) or (len(current_holdings) == 0)
        turnover = 0.0

        if rebalance_now:
            signal_date = previous_date
            scores = calculate_momentum_scores(
                adj_close,
                volume,
                daily_returns,
                signal_date,
                lookback_months,
                method,
            )

            scores = scores.replace([np.inf, -np.inf], np.nan).dropna()
            if len(scores) > 0:
                top_n = max(1, int(len(scores) * decile))
                selected = scores.sort_values(ascending=False).head(top_n)
                current_holdings = selected.index
                mom_signal = float(selected.mean()) if len(selected) > 0 else 0.0
            else:
                current_holdings = pd.Index([])
                mom_signal = 0.0

            if '^VIX' in adj_close.columns:
                vix_series = adj_close['^VIX'].loc[adj_close.index <= signal_date].dropna()
                vix_level = float(vix_series.iloc[-1]) if len(vix_series) > 0 else np.nan
            else:
                vix_level = np.nan

            trend_strength = classify_trend(adj_close, signal_date)
            sizing = calculate_position_size_v3(vix_level, mom_signal, trend_strength)
            equity_weight = sizing['equity_weight']
            cash_weight = sizing['cash_weight']

            new_asset_weights = {'CASH': cash_weight}
            if len(current_holdings) > 0 and equity_weight > 0:
                per_stock = equity_weight / len(current_holdings)
                for s in current_holdings:
                    new_asset_weights[s] = per_stock

            all_assets = set(prev_asset_weights) | set(new_asset_weights)
            l1 = sum(abs(new_asset_weights.get(a, 0.0) - prev_asset_weights.get(a, 0.0)) for a in all_assets)
            turnover = 0.5 * l1
            prev_asset_weights = new_asset_weights

        stock_rets = []
        for sym in current_holdings:
            r = _get_ret(sym, previous_date, current_date)
            if pd.notna(r) and np.isfinite(r):
                stock_rets.append(r)

        if len(stock_rets) > 0:
            stock_rets = pd.Series(stock_rets)
            if len(stock_rets) > 5:
                lo = stock_rets.quantile(winsor_q)
                hi = stock_rets.quantile(1 - winsor_q)
                stock_rets = stock_rets.clip(lo, hi)
            equity_ret = float(stock_rets.mean())
        else:
            equity_ret = 0.0

        cash_ret = _cash_return(previous_date, current_date)
        tc = (tc_bps / 10000.0) * turnover
        strategy_ret = equity_weight * equity_ret + cash_weight * cash_ret - tc

        out_index.append(current_date)
        out_rows.append({
            'strategy_returns': strategy_ret,
            'equity_returns': equity_ret,
            'cash_returns': cash_ret,
            'equity_weight': equity_weight,
            'cash_weight': cash_weight,
            'num_holdings': int(len(current_holdings)),
            'turnover': turnover,
        })

    out = pd.DataFrame(out_rows, index=pd.DatetimeIndex(out_index))
    out = out.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return out


def backtest_strategy_v5(
    adj_close,
    volume,
    lookback_months=6,
    method='simple',
    decile=0.3,
    tc_bps=5,
    holding_period=1,
    winsor_q=0.01,
):
    daily_returns = adj_close.pct_change(fill_method=None)

    month_ends = adj_close.resample('M').last().index
    valid_month_ends = []
    for me in month_ends:
        month_data = adj_close[(adj_close.index.month == me.month) & (adj_close.index.year == me.year)]
        if len(month_data) > 0:
            valid_month_ends.append(month_data.index[-1])
    valid_month_ends = pd.DatetimeIndex(valid_month_ends).unique().sort_values()

    if len(valid_month_ends) <= lookback_months + 1:
        return pd.DataFrame(columns=[
            'strategy_returns', 'equity_returns', 'cash_returns',
            'equity_weight', 'cash_weight', 'num_holdings', 'turnover'
        ])

    def _get_ret(symbol, start_date, end_date):
        if symbol not in adj_close.columns:
            return np.nan
        try:
            p0 = adj_close.at[start_date, symbol]
            p1 = adj_close.at[end_date, symbol]
            if pd.isna(p0) or pd.isna(p1) or p0 == 0:
                return np.nan
            return p1 / p0 - 1
        except Exception:
            return np.nan

    def _cash_return(prev_date, curr_date):
        if '^IRX' not in adj_close.columns:
            return 0.0
        rf = adj_close['^IRX'].loc[(adj_close.index > prev_date) & (adj_close.index <= curr_date)]
        if len(rf) == 0:
            return 0.0
        daily_rf = pd.to_numeric(rf, errors='coerce') / 100.0 / 252.0
        daily_rf = daily_rf.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        return float((1.0 + daily_rf).prod() - 1.0)

    out_rows = []
    out_index = []

    current_holdings = pd.Index([])
    equity_weight = 0.0
    cash_weight = 1.0
    prev_asset_weights = {'CASH': 1.0}
    current_asset_weights = {'CASH': 1.0}
    prev_regime = 'Normal'

    start_i = lookback_months
    for i in range(start_i, len(valid_month_ends)):
        current_date = valid_month_ends[i]
        previous_date = valid_month_ends[i - 1]

        rebalance_now = ((i - start_i) % holding_period == 0) or (len(current_holdings) == 0)
        turnover = 0.0

        if rebalance_now:
            signal_date = previous_date
            scores = calculate_momentum_scores(
                adj_close,
                volume,
                daily_returns,
                signal_date,
                lookback_months,
                method,
            )

            scores = scores.replace([np.inf, -np.inf], np.nan).dropna()
            if len(scores) > 0:
                top_n = max(1, int(len(scores) * decile))
                selected = scores.sort_values(ascending=False).head(top_n)
                current_holdings = selected.index
                mom_signal = float(selected.mean()) if len(selected) > 0 else 0.0
            else:
                current_holdings = pd.Index([])
                mom_signal = 0.0

            # V5: Dual-factor regime classification
            vix_daily = adj_close['^VIX'] if '^VIX' in adj_close.columns else pd.Series(dtype=float)
            vix3m_daily = adj_close['^VIX3M'] if '^VIX3M' in adj_close.columns else None

            regime, w_vix = compute_monthly_vix_regime(
                vix_daily, vix3m_daily, signal_date,
                prev_regime=prev_regime,
            )
            prev_regime = regime

            trend_strength = classify_trend(adj_close, signal_date)
            sizing = calculate_position_size_v5(regime, w_vix, mom_signal, trend_strength)
            equity_weight = sizing['equity_weight']
            cash_weight = sizing['cash_weight']

            weights = {}

            # V4 Change 4: momentum-weighted stock allocation
            if len(current_holdings) > 0 and equity_weight > 0 and len(scores) > 0:
                held_scores = scores.loc[scores.index.isin(current_holdings)]
                if len(held_scores) > 0:
                    shifted = held_scores - held_scores.min() + 0.01
                    if float(shifted.sum()) > 0:
                        stock_weights = (shifted / shifted.sum()) * equity_weight
                        for ticker in current_holdings:
                            if ticker in stock_weights.index:
                                weights[ticker] = float(stock_weights[ticker])
                            else:
                                weights[ticker] = equity_weight / len(current_holdings)
                    else:
                        per_stock = equity_weight / len(current_holdings)
                        for ticker in current_holdings:
                            weights[ticker] = per_stock
                else:
                    per_stock = equity_weight / len(current_holdings)
                    for ticker in current_holdings:
                        weights[ticker] = per_stock

            # V4 Change 5: conditional short leg in downtrend regimes
            if trend_strength == 'Downtrend' and regime != 'Panic' and len(scores) > 0:
                n_short = max(1, int(len(scores) * 0.10))
                short_tickers = scores.nsmallest(n_short).index.tolist()
                short_weight = 0.15 * cash_weight
                if len(short_tickers) > 0:
                    per_short = short_weight / len(short_tickers)
                    for t in short_tickers:
                        weights[t] = weights.get(t, 0.0) - per_short

            new_asset_weights = {'CASH': cash_weight}
            for k, v in weights.items():
                if pd.notna(v) and np.isfinite(v):
                    new_asset_weights[k] = float(v)

            all_assets = set(prev_asset_weights) | set(new_asset_weights)
            l1 = sum(abs(new_asset_weights.get(a, 0.0) - prev_asset_weights.get(a, 0.0)) for a in all_assets)
            turnover = 0.5 * l1
            prev_asset_weights = new_asset_weights
            current_asset_weights = new_asset_weights

        # Realized returns for all non-cash positions (long and short weights)
        realized = {}
        for sym, w in current_asset_weights.items():
            if sym == 'CASH':
                continue
            r = _get_ret(sym, previous_date, current_date)
            if pd.notna(r) and np.isfinite(r):
                realized[sym] = float(r)

        if len(realized) > 0:
            ret_series = pd.Series(realized)
            if len(ret_series) > 5:
                lo = ret_series.quantile(winsor_q)
                hi = ret_series.quantile(1 - winsor_q)
                ret_series = ret_series.clip(lo, hi)

            common = [s for s in ret_series.index if s in current_asset_weights and s != 'CASH']
            if len(common) > 0:
                w_series = pd.Series({s: current_asset_weights[s] for s in common}, dtype=float)
                weighted_stock_ret = float((w_series * ret_series.loc[common]).sum())
                gross_exposure = float(w_series.abs().sum())
                equity_ret = float(weighted_stock_ret / gross_exposure) if gross_exposure > 0 else 0.0
            else:
                weighted_stock_ret = 0.0
                equity_ret = 0.0
        else:
            weighted_stock_ret = 0.0
            equity_ret = 0.0

        cash_ret = _cash_return(previous_date, current_date)
        tc = (tc_bps / 10000.0) * turnover
        strategy_ret = weighted_stock_ret + cash_weight * cash_ret - tc

        out_index.append(current_date)
        out_rows.append({
            'strategy_returns': strategy_ret,
            'equity_returns': equity_ret,
            'cash_returns': cash_ret,
            'equity_weight': equity_weight,
            'cash_weight': cash_weight,
            'num_holdings': int(len(current_holdings)),
            'turnover': turnover,
        })

    out = pd.DataFrame(out_rows, index=pd.DatetimeIndex(out_index))
    out = out.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return out

In [ ]:
# Reload data
adj_close = pd.read_csv('adj_close.csv', index_col=0, parse_dates=True)
volume = pd.read_csv('volume.csv', index_col=0, parse_dates=True)

if adj_close.index.tz is not None:
    adj_close.index = adj_close.index.tz_localize(None)
if volume.index.tz is not None:
    volume.index = volume.index.tz_localize(None)

ret_simple_v5 = backtest_strategy_v5(adj_close, volume, lookback_months=6, method='simple', decile=0.3, holding_period=1)
ret_risk_v5 = backtest_strategy_v5(adj_close, volume, lookback_months=6, method='risk_adjusted', decile=0.3, holding_period=1)
ret_vol_v5 = backtest_strategy_v5(adj_close, volume, lookback_months=6, method='volume_weighted', decile=0.3, holding_period=1)

for name, df_ in [
    ('Simple', ret_simple_v5),
    ('Risk-Adjusted', ret_risk_v5),
    ('Volume-Weighted', ret_vol_v5),
]:
    r = df_['strategy_returns'] if 'strategy_returns' in df_.columns else pd.Series(dtype=float)
    ann_ret = (1 + r.mean()) ** 12 - 1 if len(r) > 0 else 0.0
    ann_vol = r.std() * np.sqrt(12) if len(r) > 1 else np.nan
    sharpe = ann_ret / ann_vol if pd.notna(ann_vol) and ann_vol > 0 else np.nan
    print(f"{name}: months={len(r)}, ann_ret={ann_ret:.2%}, ann_vol={ann_vol:.2%}, sharpe={sharpe:.3f}")

### Holding Period Analysis

In [ ]:
def analyze_holding_periods(adj_close, volume, backtest_fn, methods=None, holding_periods=None, lookback_months=6):
    if methods is None:
        methods = ['simple', 'risk_adjusted', 'volume_weighted']
    if holding_periods is None:
        holding_periods = [1, 2, 3, 6]

    train_end = pd.Timestamp('2024-12-31')
    rows = []

    for m in methods:
        for hp in holding_periods:
            bt = backtest_fn(
                adj_close,
                volume,
                lookback_months=lookback_months,
                method=m,
                decile=0.3,
                tc_bps=5,
                holding_period=hp,
            )
            r = bt['strategy_returns']
            r_train = r[r.index <= train_end]
            if len(r_train) < 12:
                continue

            ann_ret = (1 + r_train.mean()) ** 12 - 1
            ann_vol = r_train.std() * np.sqrt(12)
            sharpe = ann_ret / ann_vol if ann_vol > 0 else np.nan
            max_dd = ((1 + r_train).cumprod() / (1 + r_train).cumprod().cummax() - 1).min()

            rows.append({
                'method': m,
                'holding_period': hp,
                'months': len(r_train),
                'annual_return': ann_ret,
                'annual_volatility': ann_vol,
                'sharpe': sharpe,
                'max_drawdown': max_dd,
            })

    return pd.DataFrame(rows).sort_values('sharpe', ascending=False)

In [ ]:
holding_period_results_v5 = analyze_holding_periods(adj_close, volume, backtest_strategy_v5)
holding_period_results_v3 = analyze_holding_periods(adj_close, volume, backtest_strategy_v3)

print("V5 holding period grid:")
print(holding_period_results_v5.round(4))
print("\nV3 holding period grid:")
print(holding_period_results_v3.round(4))

best_config_v5 = holding_period_results_v5.iloc[0]
best_config_v3 = holding_period_results_v3.iloc[0]

print("\nBest V5 config:")
print(best_config_v5)
print("\nBest V3 config:")
print(best_config_v3)

best_strategy_v5_df = backtest_strategy_v5(
    adj_close,
    volume,
    lookback_months=6,
    method=best_config_v5['method'],
    decile=0.3,
    tc_bps=5,
    holding_period=int(best_config_v5['holding_period']),
)

best_strategy_v3_df = backtest_strategy_v3(
    adj_close,
    volume,
    lookback_months=6,
    method=best_config_v3['method'],
    decile=0.3,
    tc_bps=5,
    holding_period=int(best_config_v3['holding_period']),
)

best_v5_returns = best_strategy_v5_df['strategy_returns']
best_v3_returns = best_strategy_v3_df['strategy_returns']

In [ ]:
print("Momentum decay analysis (score vs forward returns)")

daily_returns = adj_close.pct_change(fill_method=None)
month_ends = adj_close.resample('M').last().index
valid_month_ends = []
for me in month_ends:
    m = adj_close[(adj_close.index.month == me.month) & (adj_close.index.year == me.year)]
    if len(m) > 0:
        valid_month_ends.append(m.index[-1])
valid_month_ends = pd.DatetimeIndex(valid_month_ends).unique().sort_values()

score_forward_corr = {}
for h in [1, 2, 3, 4, 5, 6]:
    corr_list = []
    for i in range(6, len(valid_month_ends) - h):
        d0 = valid_month_ends[i]
        d1 = valid_month_ends[i + h]
        scores = calculate_momentum_scores(adj_close, volume, daily_returns, d0, 6, method='simple')
        if len(scores) == 0:
            continue

        fwd = (adj_close.loc[d1, scores.index] / adj_close.loc[d0, scores.index] - 1).replace([np.inf, -np.inf], np.nan).dropna()
        common = scores.index.intersection(fwd.index)
        if len(common) >= 10:
            c = scores.loc[common].corr(fwd.loc[common])
            if pd.notna(c):
                corr_list.append(c)

    score_forward_corr[h] = float(np.mean(corr_list)) if len(corr_list) > 0 else np.nan

for h, c in score_forward_corr.items():
    print(f"Forward {h}M correlation: {c:.4f}")

print("\nAutocorrelation of strategy returns (best V5 config):")
sr = best_strategy_v5_df['strategy_returns']
for lag in [1, 2, 3, 4, 5, 6]:
    print(f"Lag {lag}: {sr.autocorr(lag=lag):.4f}")

### Performance Metrics

In [ ]:
def calculate_annual_metrics(monthly_returns, rf_monthly=None):
    monthly_returns = monthly_returns.replace([np.inf, -np.inf], np.nan).dropna()
    if len(monthly_returns) == 0:
        return {
            'months': 0,
            'annual_return': np.nan,
            'annual_volatility': np.nan,
            'sharpe_ratio': np.nan,
            'max_drawdown': np.nan,
            'win_rate': np.nan,
        }

    ann_ret = (1 + monthly_returns.mean()) ** 12 - 1
    ann_vol = monthly_returns.std() * np.sqrt(12)

    if rf_monthly is None:
        rf_ann = 0.0
    else:
        rf_aligned = rf_monthly.reindex(monthly_returns.index, method='ffill').fillna(0.0)
        rf_ann = ((1 + rf_aligned.mean()) ** 12 - 1)

    sharpe = (ann_ret - rf_ann) / ann_vol if ann_vol > 0 else np.nan
    wealth = (1 + monthly_returns).cumprod()
    max_dd = (wealth / wealth.cummax() - 1).min()

    return {
        'months': len(monthly_returns),
        'annual_return': ann_ret,
        'annual_volatility': ann_vol,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd,
        'win_rate': (monthly_returns > 0).mean(),
    }


rf_daily = adj_close['^IRX'] / 100 / 252 if '^IRX' in adj_close.columns else pd.Series(0.0, index=adj_close.index)
rf_monthly = rf_daily.resample('M').apply(lambda x: (1 + x).prod() - 1 if len(x) > 0 else 0)

xlk_monthly_returns = adj_close['XLK'].resample('M').last().pct_change(fill_method=None).dropna()
spy_monthly_returns = adj_close['SPY'].resample('M').last().pct_change(fill_method=None).dropna()

strategy_map = {
    'V5 Simple (6m, hp1)': ret_simple_v5['strategy_returns'],
    'V5 Risk-Adj (6m, hp1)': ret_risk_v5['strategy_returns'],
    'V5 Volume-Wtd (6m, hp1)': ret_vol_v5['strategy_returns'],
    'V5 Best Holding Config': best_v5_returns,
    'V3 Best Holding Config': best_v3_returns,
    'XLK Benchmark': xlk_monthly_returns,
    'SPY Benchmark': spy_monthly_returns,
}

results_summary = {}
for name, r in strategy_map.items():
    m = calculate_annual_metrics(r, rf_monthly=rf_monthly)
    results_summary[name] = m
    print(
        f"{name}: months={m['months']}, ann_ret={m['annual_return']:.2%}, "
        f"ann_vol={m['annual_volatility']:.2%}, sharpe={m['sharpe_ratio']:.3f}, "
        f"maxDD={m['max_drawdown']:.2%}, win={m['win_rate']:.1%}"
    )

In [ ]:
results_df = pd.DataFrame(results_summary).T
print(results_df.round(4))

In [ ]:
# V5 vs V3 vs Benchmarks Summary Table
comparison = pd.DataFrame({
    'V5 Best': calculate_annual_metrics(best_v5_returns, rf_monthly=rf_monthly),
    'V3 Best': calculate_annual_metrics(best_v3_returns, rf_monthly=rf_monthly),
    'XLK': calculate_annual_metrics(xlk_monthly_returns, rf_monthly=rf_monthly),
    'SPY': calculate_annual_metrics(spy_monthly_returns, rf_monthly=rf_monthly),
}).T
print("\n=== V5 vs V3 vs Benchmarks ===")
print(comparison.round(4))

## Market Regime Analysis

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from scipy.stats import ttest_1samp
import warnings
warnings.filterwarnings('ignore')

print("🔍 Market Regime Analysis - Data Preparation")
print("="*60)

# Load data and calculate benchmarks
adj_close = pd.read_csv('adj_close.csv', index_col=0, parse_dates=True)
volume = pd.read_csv('volume.csv', index_col=0, parse_dates=True)

# Handle timezone issues
if adj_close.index.tz is not None:
    adj_close.index = adj_close.index.tz_localize(None)
if volume.index.tz is not None:
    volume.index = volume.index.tz_localize(None)

# Calculate monthly data for market regime classification
print("📊 Calculating market benchmarks...")

# XLK/SPY monthly returns (market benchmarks)
# Use 'M' for pandas 1.x compatibility (pandas 2.x uses 'ME')
xlk_monthly = adj_close['XLK'].resample('M').last().pct_change(fill_method=None).dropna()

# VIX monthly average (volatility regime indicator)
vix_monthly = adj_close['^VIX'].resample('M').mean()

# Risk-free rate monthly (from IRX)
rf_daily = adj_close['^IRX'] / 100 / 252  # Convert to daily decimal
rf_monthly = rf_daily.resample('M').apply(lambda x: (1 + x).prod() - 1 if len(x) > 0 else 0)

spy_monthly = adj_close['SPY'].resample('M').last().pct_change(fill_method=None).dropna()
print(f"✅ XLK monthly returns: {len(xlk_monthly)} months")
print(f"✅ SPY monthly returns: {len(spy_monthly)} months")
print(f"✅ VIX monthly data: {len(vix_monthly)} months") 
print(f"✅ Risk-free rate monthly: {len(rf_monthly)} months")

# Get strategy returns for analysis
print("\n📈 Running strategy backtests for regime analysis...")
strategies = {}
methods = ['simple', 'risk_adjusted', 'volume_weighted']
lookbacks = [3, 6, 12]

# Train/Test split to avoid data snooping
train_end = pd.Timestamp('2024-12-31')
print(f"Train/Test split: train <= {train_end.date()}, test > {train_end.date()}")

# Test different combinations to find optimal parameters
best_sharpe = -999
best_strategy = None
best_params = None
valid_strategies = {}

for method in methods:
    for lb in lookbacks:
        try:
            strategy_returns = backtest_strategy_v5(adj_close, volume, lookback_months=lb, method=method, decile=0.3)
            strategy_returns = strategy_returns['strategy_returns']
            strategy_returns_train = strategy_returns[strategy_returns.index <= train_end]
            if len(strategy_returns_train) > 24:  # Need sufficient data for analysis
                # Calculate Sharpe ratio with improved numerical stability
                
                # Remove any extreme outliers first
                returns_clean = strategy_returns_train.copy()
                q1 = returns_clean.quantile(0.01)
                q99 = returns_clean.quantile(0.99)
                returns_clean = returns_clean.clip(q1, q99)
                
                annual_ret = returns_clean.mean() * 12
                annual_vol = returns_clean.std() * np.sqrt(12)
                
                # Align rf_monthly with strategy returns and clean
                rf_aligned = rf_monthly.reindex(returns_clean.index, method='ffill').fillna(0)
                rf_aligned = rf_aligned.clip(0, 0.1)  # Cap risk-free rate at reasonable range
                rf_annual = rf_aligned.mean() * 12
                
                # Calculate Sharpe with numerical stability checks
                if annual_vol > 0.001 and not np.isnan(annual_vol) and not np.isinf(annual_vol):
                    sharpe = (annual_ret - rf_annual) / annual_vol
                    
                    # Sanity check for Sharpe ratios
                    if -10 <= sharpe <= 10 and not np.isnan(sharpe) and not np.isinf(sharpe):
                        strategies[f"{method}_{lb}m"] = {
                            'returns': strategy_returns,
                            'sharpe': sharpe,
                            'annual_return': annual_ret,
                            'annual_volatility': annual_vol
                        }
                        valid_strategies[f"{method}_{lb}m"] = sharpe
                        
                        if sharpe > best_sharpe:
                            best_sharpe = sharpe
                            best_strategy = f"{method}_{lb}m"
                            best_params = (method, lb)
                        
                        print(f"  {method} ({lb}m): Sharpe = {sharpe:.3f}, Ann.Ret = {annual_ret:.2%}")
                    else:
                        print(f"  ❌ {method} ({lb}m): Invalid Sharpe = {sharpe:.3f}")
                else:
                    print(f"  ❌ {method} ({lb}m): Invalid volatility = {annual_vol:.6f}")
        except Exception as e:
            print(f"  ❌ {method} ({lb}m): Error - {str(e)[:50]}...")

# Fallback if no valid strategy found
if best_strategy is None and len(strategies) > 0:
    # Use the first available strategy as fallback
    best_strategy = list(strategies.keys())[0]
    best_sharpe = strategies[best_strategy]['sharpe']
    print(f"\n⚠️ Using fallback strategy due to calculation issues")

if best_strategy is not None:
    print(f"\n🏆 Best Strategy: {best_strategy} (Sharpe: {best_sharpe:.3f})")
    primary_strategy = strategies[best_strategy]['returns']
    primary_strategy = primary_strategy[primary_strategy.index > train_end]
else:
    print(f"\n❌ No valid strategy found. Creating simple 6-month strategy.")
    # Create a simple fallback strategy
    primary_strategy = backtest_strategy_v5(adj_close, volume, lookback_months=6, method='simple', decile=0.3)
    primary_strategy = primary_strategy['strategy_returns']
    primary_strategy = primary_strategy[primary_strategy.index > train_end]
    best_strategy = "simple_6m_fallback"
    
    # Calculate fallback stats
    if len(primary_strategy) > 0:
        annual_ret = primary_strategy.mean() * 12
        annual_vol = primary_strategy.std() * np.sqrt(12)
        rf_annual = rf_monthly.mean() * 12
        best_sharpe = (annual_ret - rf_annual) / annual_vol if annual_vol > 0 else 0
        print(f"Fallback strategy stats: Return={annual_ret:.2%}, Vol={annual_vol:.2%}, Sharpe={best_sharpe:.3f}")

print(f"\n✅ Strategy selection complete. Using {len(primary_strategy)} months of data.")

primary_strategy_df = backtest_strategy_v5(adj_close, volume, lookback_months=best_params[1] if best_params else 6, method=best_params[0] if best_params else 'simple', decile=0.3)
primary_strategy_df = primary_strategy_df[primary_strategy_df.index > train_end]

### Market Regime Classification with VIX

In [ ]:
print("\n🌊 Enhanced Market Regime Classification")
print("="*60)

# Align data to strategy timeline
strategy_index = primary_strategy_df.index
print(f"Strategy timeline: {strategy_index[0]} to {strategy_index[-1]} ({len(strategy_index)} months)")

# ============================================================
# 1. VOLATILITY REGIME CLASSIFICATION (VIX-based, 4 levels)
# ============================================================
# Use absolute VIX thresholds based on historical norms
# VIX < 15: Very Low (complacent market)
# VIX 15-20: Low (normal calm)
# VIX 20-25: Moderate (elevated uncertainty)
# VIX 25-30: High (fear)
# VIX > 30: Extreme (panic)

vix_aligned = vix_monthly.reindex(strategy_index, method='ffill')
vix_aligned = vix_aligned.fillna(vix_monthly.mean())

print(f"\n📊 VIX Regime Thresholds (absolute levels):")
print(f"  Very Low Vol: VIX < 15 (Complacent)")
print(f"  Low Vol: 15 ≤ VIX < 20 (Normal)")
print(f"  Moderate Vol: 20 ≤ VIX < 25 (Elevated)")
print(f"  High Vol: 25 ≤ VIX < 30 (Fear)")
print(f"  Extreme Vol: VIX ≥ 30 (Panic)")

volatility_regime = pd.cut(vix_aligned, 
                          bins=[0, 15, 20, 25, 30, np.inf], 
                          labels=['Very Low Vol', 'Low Vol', 'Moderate Vol', 'High Vol', 'Extreme Vol'])

# ============================================================
# 2. MARKET TREND CLASSIFICATION (Multi-timeframe, 5 levels)
# ============================================================
# Use both short-term (3m) and long-term (12m) momentum

# Calculate rolling returns using FULL historical data
xlk_rolling_3m_full = xlk_monthly.rolling(window=3, min_periods=3).apply(lambda x: (1 + x).prod() - 1)
xlk_rolling_6m_full = xlk_monthly.rolling(window=6, min_periods=6).apply(lambda x: (1 + x).prod() - 1)
xlk_rolling_12m_full = xlk_monthly.rolling(window=12, min_periods=12).apply(lambda x: (1 + x).prod() - 1)

# Shift to avoid look-ahead bias
xlk_rolling_3m_full = xlk_rolling_3m_full.shift(1)
xlk_rolling_6m_full = xlk_rolling_6m_full.shift(1)
xlk_rolling_12m_full = xlk_rolling_12m_full.shift(1)

# Align to strategy timeline
xlk_rolling_3m = xlk_rolling_3m_full.reindex(strategy_index, method='ffill')
xlk_rolling_6m = xlk_rolling_6m_full.reindex(strategy_index, method='ffill')
xlk_rolling_12m = xlk_rolling_12m_full.reindex(strategy_index, method='ffill')
xlk_aligned = xlk_monthly.reindex(strategy_index, method='ffill').fillna(0)

print(f"\n📈 XLK Rolling Returns (aligned to test period):")
print(f"  3-month valid: {xlk_rolling_3m.notna().sum()} / {len(xlk_rolling_3m)}")
print(f"  6-month valid: {xlk_rolling_6m.notna().sum()} / {len(xlk_rolling_6m)}")
print(f"  12-month valid: {xlk_rolling_12m.notna().sum()} / {len(xlk_rolling_12m)}")

# Market Trend Classification (based on 12-month returns with finer granularity)
# Strong Bull: > 20%
# Bull: 5% ~ 20%
# Sideways: -5% ~ 5%
# Bear: -20% ~ -5%
# Strong Bear: < -20%

def classify_trend_12m(ret_12m):
    if pd.isna(ret_12m):
        return 'Unknown'
    elif ret_12m > 0.20:
        return 'Strong Bull'
    elif ret_12m > 0.05:
        return 'Bull'
    elif ret_12m > -0.05:
        return 'Sideways'
    elif ret_12m > -0.20:
        return 'Bear'
    else:
        return 'Strong Bear'

market_regime = xlk_rolling_12m.apply(classify_trend_12m)

print(f"\n📈 Market Trend Thresholds (12-month):")
print(f"  Strong Bull: > +20%")
print(f"  Bull: +5% ~ +20%")
print(f"  Sideways: -5% ~ +5%")
print(f"  Bear: -20% ~ -5%")
print(f"  Strong Bear: < -20%")

# ============================================================
# 3. SHORT-TERM MOMENTUM CLASSIFICATION (3-month, for timing)
# ============================================================
def classify_momentum_3m(ret_3m):
    if pd.isna(ret_3m):
        return 'Unknown'
    elif ret_3m > 0.10:
        return 'Strong Up'
    elif ret_3m > 0.02:
        return 'Up'
    elif ret_3m > -0.02:
        return 'Flat'
    elif ret_3m > -0.10:
        return 'Down'
    else:
        return 'Strong Down'

short_term_momentum = xlk_rolling_3m.apply(classify_momentum_3m)

print(f"\n⚡ Short-term Momentum Thresholds (3-month):")
print(f"  Strong Up: > +10%")
print(f"  Up: +2% ~ +10%")
print(f"  Flat: -2% ~ +2%")
print(f"  Down: -10% ~ -2%")
print(f"  Strong Down: < -10%")

# ============================================================
# 4. TREND STRENGTH INDICATOR (Combining 3m, 6m, 12m)
# ============================================================
def calculate_trend_strength(row):
    """
    Count how many timeframes are positive:
    - 3 positive = Strong Trend Up
    - 2 positive = Trend Up
    - 1-2 positive = Mixed
    - 0-1 positive = Trend Down
    - 0 positive = Strong Trend Down
    """
    count = 0
    if row['xlk_3m'] > 0: count += 1
    if row['xlk_6m'] > 0: count += 1
    if row['xlk_12m'] > 0: count += 1
    
    if count == 3:
        return 'Strong Uptrend'
    elif count == 2:
        return 'Uptrend'
    elif count == 1:
        return 'Mixed'
    else:
        return 'Downtrend'

trend_df = pd.DataFrame({
    'xlk_3m': xlk_rolling_3m,
    'xlk_6m': xlk_rolling_6m,
    'xlk_12m': xlk_rolling_12m
}, index=strategy_index)

trend_strength = trend_df.apply(calculate_trend_strength, axis=1)

# ============================================================
# 5. COMBINED REGIME (Vol + Trend + Momentum)
# ============================================================
combined_regime = pd.Series(
    volatility_regime.astype(str) + " | " + market_regime.astype(str) + " | " + short_term_momentum.astype(str),
    index=strategy_index
)

# Simplified combined (Vol + Trend only)
combined_simple = pd.Series(
    volatility_regime.astype(str) + " + " + market_regime.astype(str),
    index=strategy_index
)

# ============================================================
# PRINT DISTRIBUTIONS
# ============================================================
print(f"\n" + "="*60)
print("📊 REGIME DISTRIBUTION SUMMARY")
print("="*60)

print(f"\n🌡️ Volatility Regime Distribution (5 levels):")
print(volatility_regime.value_counts().sort_index())

print(f"\n📈 Market Trend Distribution (5 levels):")
trend_order = ['Strong Bull', 'Bull', 'Sideways', 'Bear', 'Strong Bear', 'Unknown']
for trend in trend_order:
    count = (market_regime == trend).sum()
    if count > 0:
        print(f"  {trend:12s}: {count} months")

print(f"\n⚡ Short-term Momentum Distribution (5 levels):")
mom_order = ['Strong Up', 'Up', 'Flat', 'Down', 'Strong Down', 'Unknown']
for mom in mom_order:
    count = (short_term_momentum == mom).sum()
    if count > 0:
        print(f"  {mom:12s}: {count} months")

print(f"\n🔄 Trend Strength Distribution (Multi-timeframe):")
print(trend_strength.value_counts())

print(f"\n🔀 Combined Regime Distribution (Vol + Trend):")
print(combined_simple.value_counts())

# Create comprehensive regime dataframe with all new classifications
regime_data = pd.DataFrame({
    'strategy_returns': primary_strategy_df['strategy_returns'],
    'equity_weight': primary_strategy_df['equity_weight'],
    'cash_weight': primary_strategy_df['cash_weight'],
    'xlk_returns': xlk_aligned,
    'spy_returns': spy_monthly.reindex(strategy_index, method='ffill').fillna(0),
    'vix_level': vix_aligned,
    'volatility_regime': volatility_regime,
    'market_regime': market_regime,
    'short_term_momentum': short_term_momentum,
    'trend_strength': trend_strength,
    'combined_regime': combined_simple,  # Use simplified version
    'full_regime': combined_regime,  # Full 3-factor version
    'xlk_rolling_3m': xlk_rolling_3m,
    'xlk_rolling_6m': xlk_rolling_6m,
    'xlk_rolling_12m': xlk_rolling_12m
}, index=strategy_index)

regime_data = regime_data.dropna(subset=['strategy_returns', 'xlk_returns', 'vix_level'])
print(f"\n✅ Final dataset: {len(regime_data)} months of complete data")
print(f"📊 Regime dimensions: {regime_data.shape[1]} features")

### Strategy Performance by Market Regime

In [ ]:
print("\n📊 Strategy Performance by Market Regime")
print("="*60)

# 1. Performance by Volatility Regime (5 levels)
print("\n1️⃣ Performance by Volatility Regime (5 levels):")
vol_regimes_present = regime_data['volatility_regime'].dropna().unique()
vol_performance = regime_data.groupby('volatility_regime')['strategy_returns'].agg([
    'count', 'mean', 'std', 
    lambda x: (x > 0).mean(),  # win rate
    'min', 'max'
]).round(4)
vol_performance.columns = ['Months', 'Avg_Monthly_Return', 'Volatility', 'Win_Rate', 'Min_Return', 'Max_Return']
vol_performance['Annualized_Return'] = (1 + vol_performance['Avg_Monthly_Return']) ** 12 - 1
vol_performance['Annualized_Volatility'] = vol_performance['Volatility'] * np.sqrt(12)
vol_performance['Sharpe_Ratio'] = vol_performance['Annualized_Return'] / vol_performance['Annualized_Volatility']
vol_performance = vol_performance[vol_performance['Months'] > 0]  # Only show regimes with data

print(vol_performance[['Months', 'Annualized_Return', 'Annualized_Volatility', 'Sharpe_Ratio', 'Win_Rate']])

# 2. Performance by Market Trend (5 levels)
print("\n2️⃣ Performance by Market Trend (5 levels):")
market_performance = regime_data.groupby('market_regime')['strategy_returns'].agg([
    'count', 'mean', 'std', 
    lambda x: (x > 0).mean(),
    'min', 'max'
]).round(4)
market_performance.columns = ['Months', 'Avg_Monthly_Return', 'Volatility', 'Win_Rate', 'Min_Return', 'Max_Return']
market_performance['Annualized_Return'] = (1 + market_performance['Avg_Monthly_Return']) ** 12 - 1
market_performance['Annualized_Volatility'] = market_performance['Volatility'] * np.sqrt(12)
market_performance['Sharpe_Ratio'] = market_performance['Annualized_Return'] / market_performance['Annualized_Volatility']
market_performance = market_performance[market_performance['Months'] > 0]

print(market_performance[['Months', 'Annualized_Return', 'Annualized_Volatility', 'Sharpe_Ratio', 'Win_Rate']])

# 3. Performance by Short-term Momentum
print("\n3️⃣ Performance by Short-term Momentum (3-month):")
if 'short_term_momentum' in regime_data.columns:
    momentum_performance = regime_data.groupby('short_term_momentum')['strategy_returns'].agg([
        'count', 'mean', 'std', 
        lambda x: (x > 0).mean()
    ]).round(4)
    momentum_performance.columns = ['Months', 'Avg_Monthly_Return', 'Volatility', 'Win_Rate']
    momentum_performance['Annualized_Return'] = (1 + momentum_performance['Avg_Monthly_Return']) ** 12 - 1
    momentum_performance = momentum_performance[momentum_performance['Months'] > 0]
    momentum_performance = momentum_performance.sort_values('Annualized_Return', ascending=False)
    print(momentum_performance)

# 4. Performance by Trend Strength (Multi-timeframe)
print("\n4️⃣ Performance by Trend Strength (Multi-timeframe):")
if 'trend_strength' in regime_data.columns:
    trend_performance = regime_data.groupby('trend_strength')['strategy_returns'].agg([
        'count', 'mean', 'std', 
        lambda x: (x > 0).mean()
    ]).round(4)
    trend_performance.columns = ['Months', 'Avg_Monthly_Return', 'Volatility', 'Win_Rate']
    trend_performance['Annualized_Return'] = (1 + trend_performance['Avg_Monthly_Return']) ** 12 - 1
    trend_performance = trend_performance[trend_performance['Months'] > 0]
    trend_performance = trend_performance.sort_values('Annualized_Return', ascending=False)
    print(trend_performance)

# 5. Combined Regime Analysis (Vol + Trend)
print("\n5️⃣ Performance by Combined Regime (Vol + Trend):")
combined_performance = regime_data.groupby('combined_regime')['strategy_returns'].agg([
    'count', 'mean', 'std', 
    lambda x: (x > 0).mean()
]).round(4)
combined_performance.columns = ['Months', 'Avg_Monthly_Return', 'Volatility', 'Win_Rate']
combined_performance['Annualized_Return'] = (1 + combined_performance['Avg_Monthly_Return']) ** 12 - 1
combined_performance = combined_performance[combined_performance['Months'] > 0]
combined_performance = combined_performance.sort_values('Annualized_Return', ascending=False)

print(combined_performance)

# 6. Statistical Significance Tests
print("\n6️⃣ Statistical Significance Tests:")
for regime_type in regime_data['volatility_regime'].unique():
    if pd.notna(regime_type):
        subset = regime_data[regime_data['volatility_regime'] == regime_type]['strategy_returns']
        if len(subset) >= 3:  # Relaxed minimum samples
            t_stat, p_value = ttest_1samp(subset, 0)
            significance = "***" if p_value < 0.01 else "**" if p_value < 0.05 else "*" if p_value < 0.1 else ""
            print(f"  {regime_type}: t-stat={t_stat:.2f}, p-value={p_value:.4f} {significance}")

# 7. Key Insights Summary
print("\n7️⃣ Key Insights Summary:")
# Best performing regime
if len(combined_performance) > 0:
    best_regime = combined_performance['Annualized_Return'].idxmax()
    best_return = combined_performance.loc[best_regime, 'Annualized_Return']
    print(f"  🏆 Best Regime: {best_regime} (Ann. Return: {best_return:.2%})")

# Worst performing regime
if len(combined_performance) > 0:
    worst_regime = combined_performance['Annualized_Return'].idxmin()
    worst_return = combined_performance.loc[worst_regime, 'Annualized_Return']
    print(f"  ⚠️ Worst Regime: {worst_regime} (Ann. Return: {worst_return:.2%})")

# 8. Momentum Effectiveness Analysis
print("\n8️⃣ Momentum Effectiveness by Market Condition:")
print("\nCorrelation between Strategy Returns and Market Returns:")
for regime in regime_data['market_regime'].unique():
    subset = regime_data[regime_data['market_regime'] == regime]
    if len(subset) >= 3:
        corr = subset['strategy_returns'].corr(subset['xlk_returns'])
        print(f"  {regime}: {corr:.3f}")

print("\nStrategy Beta (relative to XLK) by Volatility Regime:")
for regime in regime_data['volatility_regime'].unique():
    if pd.notna(regime):
        subset = regime_data[regime_data['volatility_regime'] == regime]
        if len(subset) > 10:
            # Simple beta calculation
            covariance = subset['strategy_returns'].cov(subset['xlk_returns'])
            market_variance = subset['xlk_returns'].var()
            beta = covariance / market_variance if market_variance > 0 else 0
            print(f"  {regime}: {beta:.3f}")

# 9. Cash Allocation by Regime
print("\n9️⃣ Cash Allocation by Regime:")
if 'equity_weight' in regime_data.columns:
    cash_by_regime = regime_data.groupby('combined_regime')['equity_weight'].mean().sort_values(ascending=False)
    print(cash_by_regime.rename('avg_equity_weight'))

In [ ]:
# V5 VIX Regime Analysis
vix_daily = adj_close['^VIX'] if '^VIX' in adj_close.columns else pd.Series(dtype=float)
vix3m_daily = adj_close['^VIX3M'] if '^VIX3M' in adj_close.columns else None

# Compute regimes for all months
month_ends = adj_close.resample('M').last().index
regimes = []
prev_r = 'Normal'
for me in month_ends:
    r, w = compute_monthly_vix_regime(vix_daily, vix3m_daily, me, prev_regime=prev_r)
    regimes.append({'date': me, 'regime': r, 'w_vix': w})
    prev_r = r

regime_df = pd.DataFrame(regimes).set_index('date')
print("VIX Regime Distribution:")
print(regime_df['regime'].value_counts())
print(f"\nRegime timeline (last 12 months):")
print(regime_df.tail(12))

# If VIX3M available, show term structure stats
if vix3m_daily is not None and '^VIX3M' in adj_close.columns:
    ratio = (adj_close['^VIX'] / adj_close['^VIX3M']).dropna()
    print(f"\nVIX/VIX3M ratio stats:")
    print(f"  Mean: {ratio.mean():.3f}")
    print(f"  Median: {ratio.median():.3f}")
    print(f"  Days in backwardation (>1.0): {(ratio > 1.0).sum()} ({(ratio > 1.0).mean():.1%})")

In [ ]:
import plotly.graph_objects as go

# Regime color timeline
regime_colors = {'Normal': 'green', 'Elevated': 'orange', 'Panic': 'red'}
fig = go.Figure()

for regime_name, color in regime_colors.items():
    mask = regime_df['regime'] == regime_name
    subset = regime_df[mask]
    if len(subset) > 0:
        fig.add_trace(go.Scatter(
            x=subset.index,
            y=subset['w_vix'],
            mode='markers',
            name=regime_name,
            marker=dict(color=color, size=8),
        ))

# Add VIX line on secondary axis
vix_monthly = adj_close['^VIX'].resample('M').last() if '^VIX' in adj_close.columns else pd.Series(dtype=float)
fig.add_trace(go.Scatter(
    x=vix_monthly.index, y=vix_monthly.values,
    mode='lines', name='VIX (monthly)',
    yaxis='y2', line=dict(color='gray', dash='dot'),
))

fig.update_layout(
    title='V5 VIX Regime Classification Timeline',
    yaxis=dict(title='w_vix (Position Weight)'),
    yaxis2=dict(title='VIX Level', overlaying='y', side='right'),
    template='plotly_white',
)
fig.show()

### Visualizations

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

viz_index = best_strategy_v5_df.index.intersection(best_strategy_v3_df.index)
cum_df = pd.DataFrame({
    'V5': best_v5_returns.reindex(viz_index, fill_value=0.0),
    'V3': best_v3_returns.reindex(viz_index, fill_value=0.0),
    'XLK': xlk_monthly_returns.reindex(viz_index, method='ffill').fillna(0.0),
    'SPY': spy_monthly_returns.reindex(viz_index, method='ffill').fillna(0.0),
}).dropna()

cum = (1 + cum_df).cumprod() - 1

fig = go.Figure()
fig.add_trace(go.Scatter(x=cum.index, y=cum['V5'], mode='lines', name='V5 Best'))
fig.add_trace(go.Scatter(x=cum.index, y=cum['V3'], mode='lines', name='V3 Best'))
fig.add_trace(go.Scatter(x=cum.index, y=cum['XLK'], mode='lines', name='XLK'))
fig.add_trace(go.Scatter(x=cum.index, y=cum['SPY'], mode='lines', name='SPY'))
fig.update_layout(title='Cumulative Returns: V5 vs V3 vs XLK vs SPY', yaxis_title='Cumulative Return', template='plotly_white')
fig.show()

In [ ]:
roll_12m_v5 = (1 + best_v5_returns).rolling(12).apply(np.prod, raw=True) - 1
roll_12m_v3 = (1 + best_v3_returns).rolling(12).apply(np.prod, raw=True) - 1
roll_12m_xlk = (1 + xlk_monthly_returns.reindex(best_v5_returns.index, method='ffill').fillna(0.0)).rolling(12).apply(np.prod, raw=True) - 1
vix_monthly = adj_close['^VIX'].resample('M').last() if '^VIX' in adj_close.columns else pd.Series(index=best_v5_returns.index, dtype=float)
vix_plot = vix_monthly.reindex(best_v5_returns.index, method='ffill')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=roll_12m_v5.index, y=roll_12m_v5, mode='lines', name='V5 12M'), secondary_y=False)
fig.add_trace(go.Scatter(x=roll_12m_v3.index, y=roll_12m_v3, mode='lines', name='V3 12M'), secondary_y=False)
fig.add_trace(go.Scatter(x=roll_12m_xlk.index, y=roll_12m_xlk, mode='lines', name='XLK 12M'), secondary_y=False)
fig.add_trace(go.Scatter(x=vix_plot.index, y=vix_plot, mode='lines', name='VIX', line=dict(dash='dot')), secondary_y=True)
fig.update_layout(title='Rolling 12M Returns with VIX Overlay', template='plotly_white')
fig.update_yaxes(title_text='Rolling 12M Return', secondary_y=False)
fig.update_yaxes(title_text='VIX', secondary_y=True)
fig.show()

In [ ]:
alloc_df_v5 = best_strategy_v5_df[['equity_weight', 'cash_weight']].copy()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=alloc_df_v5.index,
    y=alloc_df_v5['equity_weight'],
    mode='lines',
    stackgroup='one',
    name='Equity Weight'
))
fig.add_trace(go.Scatter(
    x=alloc_df_v5.index,
    y=alloc_df_v5['cash_weight'],
    mode='lines',
    stackgroup='one',
    name='Cash Weight'
))
fig.update_layout(title='V5 Dynamic Allocation Over Time', yaxis_title='Portfolio Weight', template='plotly_white')
fig.show()

In [ ]:
heat_df = holding_period_results_v5.pivot(index='method', columns='holding_period', values='sharpe')

fig = go.Figure(data=go.Heatmap(
    z=heat_df.values,
    x=heat_df.columns.astype(str),
    y=heat_df.index,
    colorscale='RdYlGn',
    colorbar=dict(title='Sharpe')
))
fig.update_layout(title='V5 Holding Period Grid Search Heatmap (Sharpe)', xaxis_title='Holding Period (months)', yaxis_title='Method', template='plotly_white')
fig.show()

In [ ]:
alloc_v5 = best_strategy_v5_df[['equity_weight', 'cash_weight']].copy()
alloc_v3 = best_strategy_v3_df[['equity_weight', 'cash_weight']].copy()

common_idx = alloc_v5.index.intersection(alloc_v3.index)
alloc_v5 = alloc_v5.reindex(common_idx).ffill().fillna(0.0)
alloc_v3 = alloc_v3.reindex(common_idx).ffill().fillna(0.0)

fig = make_subplots(rows=1, cols=2, subplot_titles=('V5 Allocation', 'V3 Allocation'))
fig.add_trace(go.Scatter(x=alloc_v5.index, y=alloc_v5['equity_weight'], mode='lines', stackgroup='one', name='V5 Equity', showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=alloc_v5.index, y=alloc_v5['cash_weight'], mode='lines', stackgroup='one', name='V5 Cash', showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=alloc_v3.index, y=alloc_v3['equity_weight'], mode='lines', stackgroup='one', name='V3 Equity', showlegend=True), row=1, col=2)
fig.add_trace(go.Scatter(x=alloc_v3.index, y=alloc_v3['cash_weight'], mode='lines', stackgroup='one', name='V3 Cash', showlegend=True), row=1, col=2)
fig.update_layout(title='V5 vs V3 Allocation Comparison', yaxis_title='Portfolio Weight', template='plotly_white')
fig.show()